In [1]:
import numpy as np
from scipy.stats import dirichlet
import copy
from scipy.special import kl_div
import torch
from scipy.stats import entropy
import scipy.stats as st
import seaborn as sns


import numpy as np
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn import preprocessing

In [2]:
'''
    Data loading
'''

diabetes = load_diabetes()
x = diabetes.data
y = diabetes.target
x = preprocessing.scale(x)
# y = preprocessing.scale(y)
x=np.insert(x, 0, 1, axis=1)

dim = x.shape[1]

In [4]:
def prior(dim, s, lp):
    '''
    prior distribution is uniform distribution in the lp norm ball.
    '''
    s = np.power(s, lp)
    tmp_unif = np.random.uniform(0,s,size = dim)
    tmp_unif = np.sort(tmp_unif)
    tmp_unif = np.insert(tmp_unif, 0, 0)
    tmp_unif = np.insert(tmp_unif, dim+1, s)
    unif_ball = []
    for i in range(dim):
        sign = np.random.binomial(1,0.5) * 2 - 1
        unif_ball.append(sign * np.power((tmp_unif[i+1] - tmp_unif[i]), 1/lp))
    return np.array(unif_ball)

def priors(dim, s, N, lp):
    '''
    Concatenated prior distribution for N samples using prior function
    '''
    pri = np.zeros(shape = (N, dim))
    for i in range(N):
        pri[i, :] = prior(dim, s, lp)
    return pri

def gradient(beta, x, y, dim, batch, delta, s, lp):
    '''
    calculate the stochastic gradient with the penalized term
    '''
    f = np.zeros(shape = (dim))
    randomList=np.random.randint(0,len(y)-1,size=int(batch))
    for i in randomList:
        f=f-np.dot((y[i]-np.dot(beta,x[i])),x[i])/batch
    lp_norm = np.linalg.norm(beta, ord = lp)
    if lp_norm <= s:
        return f
    else:
#         print('reach boundary')
        g1 = np.power(np.abs(beta) / lp_norm, lp-1)
        tmp_con = (lp_norm-s) * np.multiply(g1, np.sign(beta))
        f = f + tmp_con / delta
        return f

def gradients(betas, x, y, dim, batch, delta, s, lp):
    '''
    Concatenated stochastic gradient for N samples using gradient function
    '''
    grads = np.zeros(shape = (len(betas), dim))
    for i in range(len(betas)):
        grads[i,:] = gradient(betas[i], x, y, dim, batch, delta, s, lp).reshape(-1)
    return grads

def MSE(x, data_a, data_y):
    '''
    Mean squared error using matrix calculation
    '''
    diff = data_a @ x.T - data_y.reshape(-1,1) @ np.ones(shape = (1, len(x)))
    mse = np.multiply(diff, diff)
#     mse = np.sqrt(mse)
    return np.mean(mse, axis = 0)

def MSE_old(x, data_a, data_y):
    '''
    Mean squared error using regular calculation for sanity check
    '''
    batch = len(data_y)
    mse = 0
    for i in range(batch):
        diff = np.dot(data_a[i], x) - data_y[i]
        mse += diff**2
#     mse = np.sqrt(mse)
    return mse / (batch)

# utility functions to calculate covariance between two gaussian random variable
def phi0(t, gamma):
    return np.exp(-t * gamma)

def phi1(t, gamma):
    return - 1/gamma * np.exp(-t * gamma) + 1 / gamma

def phi2(t, gamma):
    tmp = 1/gamma ** 2
    tmp = tmp * (np.exp(-gamma*t)-1)
    tmp = tmp + t/gamma
    return tmp

def covariance(t, gamma):
    '''
    coveriance between two related randomness
    '''
    e2gt = np.exp(-2*gamma*t)
    egt = np.exp(-gamma*t)
    a11 = - (1/(2*gamma)) * e2gt + 1/(2*gamma)
    a12 = e2gt / (2*gamma*gamma) - egt/(gamma**2) - 1/(2*(gamma**2)) + 1/(gamma ** 2)
    a22 = -e2gt/(2*(gamma ** 3)) + 2*egt / (gamma**3) + 1/(2 * (gamma ** 3)) - 2/(gamma ** 3) + t/(gamma ** 2)
    tmp = np.array([[a11, a12], [a12,a22]])
    return tmp

In [5]:
tmp = prior(10, 2, 0.5)
np.linalg.norm(tmp, ord = 0.5)

1.6225906155663774

In [12]:
dim = x.shape[1]
N = 100
max_iteration = 5000
batch = 50
lp = 1

beta_hist = []

# the OLS solution of the unconstrained problem
beta_ols = np.linalg.lstsq(x, y, rcond = None)[0]

# Different value of s
t_range = np.linspace(0,1,5)
# constraints list
s_list = t_range * np.linalg.norm(beta_ols, ord = lp)

delta = 0.01
gamma = 0.6

# listS to store distribution of MSE 
MSE_list_s = np.zeros(shape = (len(s_list), max_iteration+1))
std_mse_list_s = np.zeros(shape = (len(s_list), max_iteration+1))
# list to store the maximum of 1 norm over several samples
norm_list_s = np.zeros(shape = (len(s_list)))

for i in range(len(s_list)):
    beta_hist_s = []
    s = s_list[i]
    lr = s * 0.00001
    print(s)
    # initialization
    beta = priors(dim, s, N, lp)
    v = np.zeros(shape = (N, dim))
    beta_hist_s.append(beta)
    MSE_list_s[i,0] = np.mean(MSE(beta, x, y))
    std_mse_list_s[i,0] = np.std(MSE(beta, x, y))

    #update
    for ite in range(max_iteration):
        if ite % 1000 ==0 and ite != 0:
            print('iter: {}'.format(ite))
#             lr = lr *0.75
#             delta = delta * 0.9

        cov = covariance(lr, gamma)
        rand = np.random.multivariate_normal([0]*2, cov, size = (N,dim))
        rand1 = rand[:, :, 0]
        rand2 = rand[:, :, 1]
        grads = gradients(beta, x, y, dim, batch, delta, s, lp)
        noise = np.random.normal(size = (N, dim))
        
        v_new = phi0(lr, gamma) * v - phi1(lr,gamma)*grads + np.sqrt(1*gamma) * rand1
        beta = beta + phi1(lr,gamma)*v - phi2(lr,gamma)*grads + np.sqrt(1*gamma) * rand2
        v = v_new
        
        MSE_list_s[i, ite+1] = np.mean(MSE(beta, x, y))
        std_mse_list_s[i, ite+1] = np.std(MSE(beta, x, y))
        beta_hist_s.append(beta)
    beta_hist.append(beta_hist_s)
    norm_list_s[i] = np.max(np.linalg.norm(beta, axis = 1, ord = lp), axis = 0)

0.0
iter: 1000
iter: 2000
iter: 3000
iter: 4000
79.17728420816152
iter: 1000
iter: 2000
iter: 3000
iter: 4000
158.35456841632305
iter: 1000
iter: 2000
iter: 3000
iter: 4000
237.53185262448457
iter: 1000
iter: 2000
iter: 3000
iter: 4000
316.7091368326461
iter: 1000
iter: 2000
iter: 3000
iter: 4000


In [10]:
np.save('./figure/save_data/diabete_norm1_mse-SGHMC.npy',MSE_list_s)
np.save('./figure/save_data/diabete_norm1_norm-SGHMC.npy',norm_list_s)
np.save('./figure/save_data/diabete_norm1_mean-SGHMC.npy',mean_list_s)
np.save('./figure/save_data/diabete_norm1_std-SGHMC.npy',std_list_s)